<a href="https://colab.research.google.com/github/JGS2020-012/text_analysis/blob/master/calcNegaPosiScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install janome

# 必要なライブラリをインポート
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
import pathlib
import pandas as pd 

# added kamegai
import requests 
import io
import csv
import sys

In [0]:
def get_stop_words():
    stop_word_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
    stop_word_res = requests.get(stop_word_url).content
    stop_word_io = io.StringIO(stop_word_res.decode('utf-8'))
    stop_words = stop_word_io.read().split('\n')
    return stop_words

STOP_WORDS = get_stop_words()

def janome_setting():
    """
    janomeのAnalyzerインスタンス生成
    （山崎さんのプログラムを拝借）
    """  
    #Unicode正規化・半角記号除去・全角記号除去・＜＞タグ除去・改行除去
    char_filters = [UnicodeNormalizeCharFilter('NFKC'),
                    RegexReplaceCharFilter('[!/:%#\$&\?\(\)~\.=\+\-…]+', ''),
                    RegexReplaceCharFilter('[『！”＃＄％＆’（）＝～｜‘｛＋＊｝＜＞？＿－＾￥＠「；：」、。・]', ''),
                    RegexReplaceCharFilter('<.*?>', ''), 
                    RegexReplaceCharFilter('[\n|\r|\t]', '')
                   ]
    #名詞だけ取得・アルファベット小文字化・基本形
    token_filters = [
                     #POSKeepFilter(['名詞']),
                     LowerCaseFilter(),
                     ExtractAttributeFilter('base_form')]
    #設定からフィルターを作る
    a = Analyzer(char_filters=char_filters, token_filters=token_filters)
    return a

ANALYZER = janome_setting()

def before_filter(before_st):
    """
    生成ずみjanomeのAnalyzerを使って、分かち書き
    （山崎さんのプログラムを拝借）
    """  
    #文章をanalyzerに掛け処理する
    after_st = ANALYZER.analyze(before_st)
    
    #ストップワードを除去する
    #stop_words = []
    # modified kamegai
    #path = 'Japanese.txt'
    #with open(path) as f:
    #    stop_words = f.read().split('\n')
    after_st = [x for x in after_st if x not in STOP_WORDS]

    return after_st

In [0]:
def load_dict(url_file):
    """
    極性表現辞書をロードする。
    """  
    res = requests.get(url_file).content
    lists = []
    reader = csv.reader(io.StringIO(res.decode('utf-8')), delimiter='\t')
    for row in reader:
        #print(reader.line_num,row[0])
        lists.append(row)
    return lists

In [0]:
# 極性表現辞書を基にスコアリングできる状態にする。

def get_sentiment_pn_dict():
    """
    日本語評価極性辞書（用言編）
           1                  2　
    (例) ネガ（評価）	恩 着せ が まし い
    (方針)・1列目のネガ（評価）は-1、ポジ（評価）は1へ変換する。
    　    ・2列目は、文字を連結して、形態素解析を行う。
    """
    sentiment_dict = {}
    dict_content = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn')
    i = 0
    for line in dict_content:
      score = 0
      if line[0] == 'ポジ（経験）':
        score = 1
      elif line[0] == 'ネガ（経験）':
        score = -1
      elif line[0] == 'ポジ（評価）':
        score = 1
      elif line[0] == 'ネガ（評価）':
        score = -1
      else:
        print('error:　{}'.format(line[0]))
        sys.exit()
      text = line[1].replace(' ','')
      text_wakati = ' '.join(before_filter(text))
      if len(text_wakati) > 0:
        sentiment_dict[text_wakati] = score
      #print('{}, {}, {}, {}'.format(score,line[1],text_wakati,len(text_wakati)))
      i = i + 1
      if i % 1000 == 0:
        print('{} line finished!'.format(i))
    return sentiment_dict

def get_sentiment_trim_dict():
    """
    日本語評価極性辞書（名詞編）
       　   1     2      3　
    (例) ２か月   e	  〜である・になる（状態）客観
    (方針)・2列目のnは-1、pは1へ変換する。（p:ポジティブ、e:ニュートラル、n:ネガティブ）
          ・3列目に・を含む場合は分割し、～を1列目で置換する。
    """   
    dict_content = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim')
    # TODO


In [133]:
# スコアリング用辞書を取得
sentiment_dict = get_sentiment_pn_dict()
#print(sentiment_dict)

1000 line finished!
2000 line finished!
3000 line finished!
4000 line finished!
5000 line finished!


In [0]:
def calc_score(text, sdict):
  # 文字の長い順にソートした極性表現リストを取得
  dict_tmp = {}
  text_origin = text
  for sentiment, score in sdict.items():
    dict_tmp[sentiment] = len(sentiment)

  sorted_sentiments = []
  for item_list in sorted(dict_tmp.items(), key=lambda x:x[1], reverse=True):
    sorted_sentiments.append(item_list[0])
  #print(sorted_sentiments)

  i = 0
  score = 0
  while i < len(sorted_sentiments):
    if sorted_sentiments[i] in text:
      matched = sorted_sentiments[i] 
      score = score + sdict[matched]
      # 最初にマッチした極性表現部分をマスク
      if sdict[matched] == 1:
        text = text.replace(matched,'【+】',1)
      elif sdict[matched] == -1:
        text = text.replace(matched,'【-】',1)
      # 同一の極性表現が存在する可能性があるため、もう一回チェック
      i = i - 1
      #print('{}, {}, {}, {}, {}'.format(i, score, matched, text_origin, text))
    else:
      i = i + 1
  return (score,text)

In [0]:
def get_score(twitter):
  twitter_wakati = ' '.join(before_filter(twitter))
  score, mask = calc_score(twitter_wakati, sentiment_dict)
  return (score, mask)

In [136]:
# テスト
twitter = "LINEpayはなじまない。けど、payカードはなじむと思ったら、やっぱりなじんだ！"
get_score(twitter)

(1, 'linepay は 【-】 けど pay カード は 【+】 と 思う た やっぱり 【+】 だ')